![](imgs/logo.png)

# Przetwarzanie Big Data z użyciem Apache Spark

Autor notebooka: Jakub Nowacki.

## MapReduce

### Map, reduce (i filter) in Python

Nazwa **MapReduce** pochodzi od funkcji *map* i *reduce*.  

*Map* i *reduce* (czasami nazywany *fold*) są typowe dla *programowania funkcyjnego*. 

In [ ]:
# Typowa definicja funkcji w Python
def squared(x):
    return x**2

In [ ]:
squared

In [ ]:
squared(12)

In [ ]:
# funkcja anonimowa zwana też lambda
lambda x: x**2

In [ ]:
# Referencje do lambdy można przypisać do zmiennej
squared_bis = lambda x: x**2

In [ ]:
squared_bis(12)

In [ ]:
# map - wykonaj funkcje na wszystkich elementach kolekcji (iterable)
# map(function, iterable)
map(squared, [1, 2, 3, 4, 5])

In [ ]:
# map w Python 3 zwraca iterator; żeby otrzymać listę trzeba wykonać poniższe
list(map(squared, [1, 2, 3, 4, 5]))

In [ ]:
# W Python 3 reduce przesunięte zostało do functools
#from functools import reduce 

# reduce - redukuje kolekcje do jednego elementu za pomocą funkcji (np. suma, )
# reduce(function, iterable, accumulator=0)
reduce(lambda x, y: x + y, [1, 2, 3, 4, 5])

In [ ]:
# filter - filtruje kolekcje używając funkcji filtrującej zwracającej wartość boolowską
# filter(function, iterable)
filter(lambda x: x % 2 == 1, [1, 2, 3, 4, 5])

In [ ]:
# Podobnie, w Python 3 dostajemy iterator; należy użyć listy aby otrzymać kolekcję
list(filter(lambda x: x % 2 == 1, [1, 2, 3, 4, 5]))

### Zadania

Używająć `map`, `filter` i `reduce`, otrzymaj:

* Iloczyn `[1, 2, 3, 4, 5]`.
* Długość każdego słowa w liście `["Python", "Spark", "Big", "Data", "ML", "scikit-learn"]`.
* (★) Sumę wszystkich liter w słowach z powyższej listy nie zawierających litery `"i"`.

In [ ]:
print(reduce(lambda x,y: x*y,[1, 2, 3, 4, 5]))
print(list(map(lambda item: len(item),["Python", "Spark", "Big", "Data", "ML", "scikit-learn"])))
print(reduce(lambda x,y: x+y,list(map(lambda item: len(item),filter(lambda x: "i" not in x,["Python", "Spark", "Big", "Data", "ML", "scikit-learn"])))))

In [ ]:
# Przy okazji
x = range(1000000)

In [ ]:
%%timeit
y = filter(lambda x: x % 2 == 1, x)

In [ ]:
%%timeit
y = [each for each in x if each % 2 == 1]

## MapReduce in Hadoop

W Hadoop MapReduce realizowane jest z użyciem par klucz-wartość. Zobacz poniższy przykład:
![](imgs/MapReduce_example.png)

In [ ]:
import sys
from StringIO import StringIO
# Python 3
#from io import StringIO
import contextlib

# Funkcja pomocnicza przechwytująca strumień wyjściowy
@contextlib.contextmanager
def stdoutIO(stdout=None):
    old = sys.stdout
    if stdout is None:
        stdout = StringIO()
    sys.stdout = stdout
    yield stdout
    sys.stdout = old

# Linie wejściowe do przetworzenia
lines = ['123199901', '567200806', '645200811', '989199933', '452199904', '224200822']

# Mapper wyciągający rok i liczbę
def mapper(lines):
    for line in lines:
        key = int(line[3:7])
        value = int(line[7:])
        print("{0}<>{1}".format(key, value))

# Reducer liczący sumę
def reducer(lines):
    lastKey = None
    reduce_sum = 0
    for line in lines: 
        key, value = line.split("<>")
        if lastKey is None:
            lastKey = key
        if key != lastKey:
            print("{0},{1}".format(lastKey, reduce_sum))
            reduce_sum = 0

        reduce_sum += int(value)
        lastKey = key
    print("{0},{1}".format(lastKey, reduce_sum))
    
# Przebieg MapReduce 
# Input
print("Input: {}".format(lines))
# Map
with stdoutIO() as mapper_out:
    mapper(lines)
shuffled = mapper_out.getvalue().strip().split('\n')
print("Mapper out: {}".format(shuffled))
# Shuffle
shuffled.sort()
print("Shuffeled mapper out: {}".format(shuffled))
# Reduce
with stdoutIO() as reducer_out:
    reducer(shuffled)
# Output
output = reducer_out.getvalue().strip().split('\n')
print("Output: {}".format(output))



## Podstawy Spark

In [1]:
import pyspark

In [2]:
sc = pyspark.SparkContext(master='local[*]',appName="sparkMapReduce")

In [ ]:
# RDD - Resilient Distributed Datasets, rozpraszanie danych w Spark
rdd = sc.parallelize(range(10))

In [ ]:
# Obiekt a nie właściwe dane
rdd

In [ ]:
# zwraca wszystkie elementy; należy używać z rozwagą
rdd.collect()

In [ ]:
# zwraca liczbę elementów
rdd.count()

In [ ]:
# zwraca pierwszy
rdd.first()

In [ ]:
# zwraca 5 pierwszych elementów
rdd.take(5)

In [ ]:
# zwraca 5 elementów z najwyższą wartością
rdd.top(5)

In [ ]:
# bierze próbkę losową 3 elementów (bez zwracania próbek)
rdd.takeSample(False, 3)

In [ ]:
# operacje można łączyć w łańcuch
rdd.filter(lambda x: x % 2 == 0)\
    .map(lambda x: x*x*x)\
    .collect()

In [ ]:
# suma wszystkich elementów
rdd.reduce(lambda x, y: x + y)

In [ ]:
# alternatywnie możemy wykorzystać operator
from operator import add
rdd.reduce(add)

In [ ]:
# kolejny przykład z liczeniem słów
animals = sc.parallelize(["cat", "python", "cat", "snake", "snake"])

In [ ]:
# mapa do pary klucz-wartość
animal_kv = animals.map(lambda x: (x, 1))

In [ ]:
animal_kv.collect()

In [ ]:
# redukujemy parami wszystkie wartości dla danego klucza
animal_kv \
  .reduceByKey(add)  \
  .collect()

## Text processing

Będziemy analizować ["The Tragedy of Titus Andronicus" by William Shakespeare](http://www.gutenberg.org/cache/epub/1106/pg1106.txt) z Project Gutenberg.

Zbiór nie jest Big Data ale ilustruje koncepcję przetwarzania w Spark.

In [7]:
lines = sc.textFile("data/titus_andronicus.txt")

In [8]:
# liczba linii
lines.count()

3255

In [9]:
lines.take(5)

[u'',
 u'This Etext file is presented by Project Gutenberg, in',
 u'cooperation with World Library, Inc., from their Library of the',
 u'Future and Shakespeare CDROMS.  Project Gutenberg often releases',
 u'Etexts that are NOT placed in the Public Domain!!']

In [10]:
# zmieniamy linie w kolekcje słów
words = lines.flatMap(lambda x: x.split())

In [11]:
words.take(5)

[u'This', u'Etext', u'file', u'is', u'presented']

In [12]:
# liczba słów
words.count()

23531

### Zadanie

* Co się stanie jak użyjemy `map` zamiast `flatMap`?

In [13]:
# Liczymy słowa tylko z pierwszą wielką literą
capitalized = words \
  .filter(lambda x: x[0].isupper()) \

In [14]:
capitalized.take(5)

[u'This', u'Etext', u'Project', u'Gutenberg,', u'World']

In [15]:
capitalized.distinct().take(10)

[u'REMEDIES',
 u'Saturninus',
 u'GOTHS',
 u'Soft!',
 u'Fame,',
 u'Somewhither',
 u'Saint',
 u'Publius,',
 u'Andronicus',
 u'Pyramus']

In [17]:
from operator import add
capitalized \
  .map(lambda x: (x, 1)) \
  .reduceByKey(add) \
  .top(10, lambda x: x[1])  # możemy wybrać wartość z pary klucz-wartość jako element sortujący

[(u'I', 390),
 (u'And', 289),
 (u'TITUS.', 118),
 (u'The', 90),
 (u'To', 85),
 (u'That', 84),
 (u'MARCUS.', 65),
 (u'But', 61),
 (u'AARON.', 58),
 (u'For', 55)]

### Zadania

* Wypisz 5 linii zaczynających się od "Titus" lub "Marcus" (usuwając spacje jeżeli potrzeba).
* Wypisz 20 najpopularniejszych słów z samymi WIELKIMI LITERAMI.
* ★ Jaka jestczęstotliwość wyrazów w dziele?

### (Python) hints

In [27]:
lines.filter(lambda x: x.strip().startswith('Titus') or x.strip().startswith('Marcus')).take(3)

[u'    Titus, unkind, and careless of thine own,',
 u'    Titus Andronicus, the people of Rome,',
 u'    Marcus, unknit that sorrow-wreathen knot;']

In [35]:
# zmieniamy linie w kolekcje słów
words = lines.flatMap(lambda x: x.split())
capitalized = words \
  .filter(lambda x: x.isupper()) \

capitalized \
  .map(lambda x: (x, 1)) \
  .reduceByKey(add) \
  .top(20, lambda x: x[1])  # możemy wybrać wartość z pary klucz-wartość jako element sortujący

[(u'I', 390),
 (u'TITUS.', 118),
 (u'MARCUS.', 65),
 (u'AARON.', 58),
 (u'LUCIUS.', 51),
 (u'SATURNINUS.', 50),
 (u'TAMORA.', 49),
 (u'A', 45),
 (u'DEMETRIUS.', 39),
 (u'O,', 37),
 (u'OF', 36),
 (u'OR', 36),
 (u'FOR', 32),
 (u'CHIRON.', 30),
 (u'O', 30),
 (u'AND', 27),
 (u'BY', 27),
 (u'ARE', 22),
 (u'IS', 20),
 (u'ELECTRONIC', 18)]

In [38]:
words = lines.flatMap(lambda x: x.split())
words \
    .map(lambda x: (x,1)) \
    .reduceByKey(add) \
    .top(20, lambda x: x[1])

[(u'the', 633),
 (u'and', 562),
 (u'to', 419),
 (u'I', 390),
 (u'of', 348),
 (u'And', 289),
 (u'my', 273),
 (u'a', 269),
 (u'in', 259),
 (u'with', 256),
 (u'that', 195),
 (u'you', 190),
 (u'thy', 189),
 (u'for', 172),
 (u'this', 165),
 (u'is', 156),
 (u'his', 154),
 (u'not', 150),
 (u'thou', 148),
 (u'be', 145)]

In [ ]:
"  some string with whitespaces \t  ".strip()

In [ ]:
"Jake likes his dog.".startswith("Anne")

In [18]:
"Jake likes his dog.".startswith("Jake")

True

In [ ]:
"Anne" or "Jake"  # Nie rób: string.startswith(a or b)

In [ ]:
"abc,-".replace(",", "")

In [ ]:
"abc,-".replace(",", "").replace("-", "")

In [ ]:
# Wyrażenia regularne w pythonie
import re
re.findall("[\w]+", "Titus Andronicus Roman-legion")